# Edit Tabular Data
Edit and compare tabular data using the Python `numpy` library.

In this example, there is some existing tensile data for a material (AMS 6520, Plate, 1000°F).
Some of that data is of particular interest and we want to separate it from the old data and
put it in a record of its own.

## Connect to MI
Specify a database and table.

In [1]:
from datetime import datetime
import numpy as np
import sys
from GRANTA_MIScriptingToolkit import granta as mpy

mi = mpy.connect('http://localhost/mi_servicelayer', autologon=True)
dbase = mi.get_db(db_key='MI_Training')
table = dbase.get_table('Tensile Statistical Data')

## Locate existing data
Find the record and attribute that contains the original data.

In [2]:
record = table.search_for_records_by_name('AMS 6520, Plate, 1000°F')[0]
samples = record.attributes['Tensile test data used in this rollup']

Select some samples of interest.

In [3]:
focus_samples = ['MTS-615731', 'MTS-615741', 'MTS-615771']

Get their **Record** objects from the tabular attribute containing the tensile test data.

In [4]:
focus_recs = [samples.linked_records[s][0] for s in focus_samples]

Get their row indices in the tabular data structure.

In [5]:
focus_rows = [samples['Linking Value (Specimen ID)'].index(samp) for samp in focus_samples]

## Extract and analyse the data
Extract the sample data, and establish how different these particular samples are from the remainder of the set.

In [6]:
focus_youngs = [samples["Young's Modulus (11-axis)", i] for i in focus_rows]
rmain_youngs = [samples["Young's Modulus (11-axis)", i] for i in range(6) if i not in focus_rows]

rmain_mean = np.mean(rmain_youngs)
focus_mean = np.mean(focus_youngs)
rmain_mean, focus_mean

(142.74677022298178, 146.3169097900391)

In [7]:
print('Percentage difference between the two means is: '
      '{} %'.format(100*abs(rmain_mean-focus_mean)/(0.5*(rmain_mean+focus_mean))))

Percentage difference between the two means is: 2.470140535743884 %


## Create a new record and tabular attribute
Create a new record for your data of interest.

In [8]:
now = datetime.now().strftime("%c")
recordName = 'STK Example 9:{}'.format(now)
new_rec = table.create_record(recordName, parent=record.parent, subsets={'Statistical Test Data'})

Select the tabular attribute to write to.

In [9]:
new_samples = new_rec.attributes['Tensile test data used in this rollup']
new_samples

<TabularValue name: Tensile test data used in this rollup (not loaded)>

Link the samples to the new tabular datum.

In [10]:
for focus_sample in focus_samples:
    new_samples.add_row(linking_value=focus_sample)
new_samples

<TabularValue name: Tensile test data used in this rollup, shape: 9 x 3>

Set the new tabular attribute to update, and write the new record to MI.

In [11]:
new_rec.set_attributes([new_samples])
new_rec = mi.update([new_rec])[0]

## Link to the original record
The original record is linked to the records you just linked to the tabular data. Link the new record to the original, too.

(Records must exist on the server to be linked together, which means the record must be pushed to the server before it can be linked to other records.)

In [12]:
new_rec.set_links('Tensile Test Data', set(focus_recs))
new_rec.links

{'Tensile Test Data': {<Record long name:MTS-615731>,
  <Record long name:MTS-615741>,
  <Record long name:MTS-615771>}}

## Update the new record's links on the server
Like changes to data, changes to links also need to be pushed to the server. Unlike attributes, links do not need to be flagged for update (there is no equivalent to `set_attributes()` for links).

In [13]:
mi.update_links([new_rec])

[<Record long name:STK Example 9:Thu Jul 15 10:45:21 2021>]